In [34]:
import pandas as pd
from sqlalchemy import create_engine
from random import randint

In [12]:
moves_file = "./Resources/move-data.csv"
pokemon_file = "./Resources/pokemon-data.csv"

In [16]:
moves_df = pd.read_csv(moves_file)
pokemon_df = pd.read_csv(pokemon_file, sep=';')

In [15]:
moves_df.head()

,Index,Name,Type,Category,Contest,PP,Power,Accuracy,Generation
0,1,Pound,Normal,Physical,Tough,35,40,100,1
1,2,Karate Chop,Fighting,Physical,Tough,25,50,100,1
2,3,Double Slap,Normal,Physical,Cute,10,15,85,1
3,4,Comet Punch,Normal,Physical,Tough,15,18,85,1
4,5,Mega Punch,Normal,Physical,Tough,20,80,85,1


In [17]:
pokemon_df.head()

,Name,Types,Abilities,Tier,HP,Attack,Defense,Special Attack,Special Defense,Speed,Next Evolution(s),Moves
0,Abomasnow,"['Grass', 'Ice']","['Snow Warning', 'Soundproof']",PU,90,92,75,92,85,60,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le..."
1,Abomasnow-Mega,"['Grass', 'Ice']",['Snow Warning'],NUBL,90,132,105,132,105,30,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le..."
2,Abra,['Psychic'],"['Inner Focus', 'Magic Guard', 'Synchronize']",LC,25,20,15,105,55,90,['Kadabra'],"['Teleport', 'Ally Switch', 'Barrier', 'Encore..."
3,Absol,['Dark'],"['Justified', 'Pressure', 'Super Luck']",PU,65,130,60,75,60,75,[],"['Perish Song', 'Future Sight', 'Me First', 'R..."
4,Absol-Mega,['Dark'],['Magic Bounce'],RUBL,65,150,60,115,60,115,[],"['Perish Song', 'Future Sight', 'Me First', 'R..."


### Transform Pokemon DataFrame

In [22]:
# List conversion function. Let me know if there are any issues
def convert(s):
    lst = []
    temp = str(s).split(", ")
    
    for x in temp:
        lst.append(x.translate({ord(i): None for i in "[']"}))
    
    return lst

In [23]:
pokemon_lst_df = pokemon_df.copy()
for i in range(0, len(pokemon_lst_df)):
    pokemon_lst_df.at[i,'Types'] = convert(pokemon_df.loc[i,'Types'])
    pokemon_lst_df.at[i,'Moves'] = convert(pokemon_df.loc[i,'Moves'])

In [75]:
pokemon_cols = ["Name", "Tier", "HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed"]
pokemon_trans_df = pokemon_lst_df[pokemon_cols].copy()
pokemon_trans_df = pokemon_trans_df.rename(columns={"Special Attack": "Special_Attack", "Special Defense": "Special_Defense"})

pokemon_trans_df['Type_1'] = ''
pokemon_trans_df['Type_2'] = 'None'
# Rename the column headers
for index, row in pokemon_lst_df.iterrows():
    for i in range(0, len(row['Types'])):
        pokemon_trans_df.loc[index,f'Type_{i+1}'] = row['Types'][i]
    pokemon_trans_df.loc[index,'Move'] = row['Moves'][randint(0,len(row['Moves'])-1)]

pokemon_trans_df.head()

,Name,Tier,HP,Attack,Defense,Special_Attack,Special_Defense,Speed,Type_1,Type_2,Move
0,Abomasnow,PU,90,92,75,92,85,60,Grass,Ice,Blizzard
1,Abomasnow-Mega,NUBL,90,132,105,132,105,30,Grass,Ice,Avalanche
2,Abra,LC,25,20,15,105,55,90,Psychic,None,Charge Beam
3,Absol,PU,65,130,60,75,60,75,Dark,None,Dream Eater
4,Absol-Mega,RUBL,65,150,60,115,60,115,Dark,None,Future Sight


### Transform Pokemon DataFrame

In [69]:
moves_cols = ["Name", "Type", "Category", "PP", "Power", "Accuracy"]
moves_trans_df = moves_df[moves_cols].copy()
moves_trans_df = moves_trans_df.rename(columns={"Name": "Move", "Type": "Move_Type"})

### Join Dataframes on Move

In [76]:
combined_df = pokemon_trans_df.merge(moves_trans_df, how = 'left', on = "Move")
combined_df.set_index('Name', inplace = True)
combined_df.head()

,Tier,HP,Attack,Defense,Special_Attack,Special_Defense,Speed,Type_1,Type_2,Move,Move_Type,Category,PP,Power,Accuracy
Name,,,,,,,,,,,,,,,
Abomasnow,PU,90,92,75,92,85,60,Grass,Ice,Blizzard,Ice,Special,5.0,110,None
Abomasnow-Mega,NUBL,90,132,105,132,105,30,Grass,Ice,Avalanche,Ice,Physical,10.0,60,100
Abra,LC,25,20,15,105,55,90,Psychic,None,Charge Beam,Electric,Special,10.0,50,90
Absol,PU,65,130,60,75,60,75,Dark,None,Dream Eater,Psychic,Special,15.0,100,100
Absol-Mega,RUBL,65,150,60,115,60,115,Dark,None,Future Sight,Psychic,Special,10.0,120,None


### Create database connection

In [77]:
connection_string = "postgres:Gabriel(00)@localhost:5432/Pokemon"
engine = create_engine(f'postgresql://{connection_string}')

In [78]:
# Confirm tables
engine.table_names()

['pokemon']

### Load DataFrames into database

In [79]:
combined_df.to_sql(name='pokemon', con=engine, if_exists='append', index=True)